In [1]:
from numba import njit, prange
import os
import glob
import numpy as np
import pandas as pd
from typing import List
from sklearn.impute import KNNImputer
from joblib import Parallel, delayed

# ────────────────────────────────────────────────────────────────
# 0. 설정 파라미터
# ────────────────────────────────────────────────────────────────
SEED: int            = 42          # 고정 시드
MISSING_RATE: float  = 0.2        # 5 % 결측치 비율
VOLT_DROP_THR: float = 3.3         # 3.3 V 미만 → NaN
NAN_DAY_THR: int     = 1400        # 하루 NaN 허용 개수 상한
VALID_VOLT_THR: float = 3.6        # 3.6 V 이상 유효 전압 기준
VALID_COUNT_REQ: int = 400         # 날짜별(분*채널) 유효 샘플 수

# 입력 ESS 폴더 (예시)
ESS_SRC_DIR = 'preprocessing_A/3.A.R5M7C11'
# 출력 루트 폴더 (결측 5 %)
ROOT_OUTPUT_DIR = 'A_20%'

# ────────────────────────────────────────────────────────────────
# KNN 임퓨테이션 (날짜별 병렬)
# ────────────────────────────────────────────────────────────────
imputer = KNNImputer(n_neighbors=5, weights='uniform')

@njit(parallel=True, cache=True)
def _sigmoid_cosine_fill(arr: np.ndarray, interp: np.ndarray, k: float, max_win: int) -> np.ndarray:
    N = arr.size
    out = np.empty_like(arr)
    for i in prange(N):
        if np.isnan(arr[i]):
            # 양쪽 2개 값 탐색
            lv = np.empty(2, np.float64)
            rv = np.empty(2, np.float64)
            lc = rc = 0
            j = i - 1
            while j >= 0 and lc < 2 and (i - j) <= max_win:
                if not np.isnan(arr[j]): lv[lc], lc = arr[j], lc+1
                j -= 1
            j = i + 1
            while j < N and rc < 2 and (j - i) <= max_win:
                if not np.isnan(arr[j]): rv[rc], rc = arr[j], rc+1
                j += 1
            if lc == 2 and rc == 2:
                nl = np.hypot(lv[0], lv[1])
                nr = np.hypot(rv[0], rv[1])
                cos_t = (lv[0]*rv[0] + lv[1]*rv[1]) / (nl * nr)
                w = 1.0 / (1.0 + np.exp(k * cos_t))
                ext = 0.5 * ((2*lv[0] - lv[1]) + (2*rv[0] - rv[1]))
                out[i] = (1 - w) * interp[i] + w * ext
            else:
                out[i] = interp[i]
        else:
            out[i] = arr[i]
    return out

@njit(parallel=True, cache=True)
def fill_all_channels(arr2d: np.ndarray, interp2d: np.ndarray, k: float, max_win: int) -> np.ndarray:
    rows, cols = arr2d.shape
    result = np.empty_like(arr2d)
    for c in prange(cols):
        result[:, c] = _sigmoid_cosine_fill(arr2d[:, c], interp2d[:, c], k, max_win)
    return result

# 워밍업: JIT 컴파일 및 캐시
_dummy = np.zeros((1,1), dtype=np.float64)
_dummy = fill_all_channels(_dummy, _dummy, k=4.0, max_win=1440)


def inject_random_missing(df: pd.DataFrame, cols: List[str], rate: float, seed: int = SEED) -> pd.DataFrame:
    rng = np.random.default_rng(seed)
    mask = rng.random(df[cols].shape) < rate
    df2 = df.copy()
    df2.loc[:, cols] = df2[cols].mask(mask)
    return df2


def preprocess_ess_folder(ess_dir: str, root_out_dir: str):
    # CSV 파일 목록
    csv_list = glob.glob(os.path.join(ess_dir, '*.csv'))
    if not csv_list:
        print(f'[WARN] No CSV in {ess_dir}')
        return

    ess_name = os.path.basename(ess_dir.rstrip('/'))
    out_dir = os.path.join(root_out_dir, ess_name)
    os.makedirs(out_dir, exist_ok=True)

    for path in csv_list:
        df = pd.read_csv(path)
        # 날짜 통합
        if 'colec_dt' not in df.columns:
            if 'clct_dt' in df.columns: df.rename(columns={'clct_dt':'colec_dt'}, inplace=True)
            else:
                print(f'[SKIP] {path} : no date')
                continue
        # 컬럼명 정규화
        rename = {c: f'cell_volt_{int(c.split("_")[-1])}' for c in df.columns if c.startswith('cel_volt_')}
        if rename: df.rename(columns=rename, inplace=True)

        # 타임라인 병합
        df['colec_dt'] = pd.to_datetime(df['colec_dt'])
        t0, t1 = df['colec_dt'].min(), df['colec_dt'].max()
        timeline = pd.DataFrame({'colec_dt': pd.date_range(t0, t1, freq='min')})
        merged = timeline.merge(df, on='colec_dt', how='left')
        if 'Unnamed: 0' in merged.columns: merged.drop('Unnamed: 0', axis=1, inplace=True)
        cols = [c for c in merged.columns if c.startswith('cell_volt_')]
        merged.loc[merged['cell_volt_1'] < VOLT_DROP_THR, cols] = np.nan

        merged['date'] = merged['colec_dt'].dt.date
        bad = merged.groupby('date')[cols].apply(lambda x: (x.isna().sum()>NAN_DAY_THR).any())
        clean = merged[~merged['date'].isin(bad[bad].index)].reset_index(drop=True)

        clean = inject_random_missing(clean, cols, MISSING_RATE)

        # 6. 날짜별 KNN + Sigmoid·Cosine 병렬
        col_idx = [clean.columns.get_loc(c) for c in cols]
        data = clean[cols].to_numpy(dtype=np.float64)
        groups = list(clean.groupby('date').groups.items())  # [(date, idx_list)]

        def process_day(idx_list):
            arr = data[idx_list]
            knn = imputer.fit_transform(arr)
            filled = fill_all_channels(arr, knn, k=4.0, max_win=1440)
            return idx_list, filled

        results = Parallel(n_jobs=-1)(delayed(process_day)(idx) for _, idx in groups)
        for idx_list, filled in results:
            clean.iloc[idx_list, col_idx] = filled.astype(np.float32)

        # 7~9: 유효 날짜 필터 및 저장
        valid = clean.groupby('date')[cols].apply(lambda x: (x>=VALID_VOLT_THR).sum().sum())
        keep = valid[valid>=VALID_COUNT_REQ*len(cols)].index
        df2 = clean[clean['date'].isin(keep)].reset_index(drop=True)
        if df2.empty:
            print(f'[WARN] {path} : no valid dates')
            continue
        first, last = df2['date'].iloc[0], df2['date'].iloc[-1]
        df2 = df2[(df2['date']!=first)&(df2['date']!=last)].reset_index(drop=True)
        if df2.empty:
            print(f'[WARN] {path} : nothing after edge')
            continue

        uniq = df2['date'].unique()
        base = os.path.splitext(os.path.basename(path))[0]
        for c in cols:
            arr = np.stack([df2[df2['date']==d][c].to_numpy(dtype=np.float32) for d in uniq])
            np.save(os.path.join(out_dir, f'{base}_{c}.npy'), arr)
        print(f'[OK] {path} → {out_dir}')

if __name__ == '__main__':
    np.random.seed(SEED)
    os.makedirs(ROOT_OUTPUT_DIR, exist_ok=True)
    preprocess_ess_folder(ESS_SRC_DIR, ROOT_OUTPUT_DIR)


[OK] preprocessing_A/3.A.R5M7C11/rack4_module1.csv → A_20%/3.A.R5M7C11
[OK] preprocessing_A/3.A.R5M7C11/rack6_module8.csv → A_20%/3.A.R5M7C11
[OK] preprocessing_A/3.A.R5M7C11/rack4_module2.csv → A_20%/3.A.R5M7C11
[OK] preprocessing_A/3.A.R5M7C11/rack6_module10.csv → A_20%/3.A.R5M7C11
[OK] preprocessing_A/3.A.R5M7C11/rack2_module13.csv → A_20%/3.A.R5M7C11
[OK] preprocessing_A/3.A.R5M7C11/rack6_module14.csv → A_20%/3.A.R5M7C11
[OK] preprocessing_A/3.A.R5M7C11/rack2_module5.csv → A_20%/3.A.R5M7C11
[OK] preprocessing_A/3.A.R5M7C11/rack5_module10.csv → A_20%/3.A.R5M7C11
[OK] preprocessing_A/3.A.R5M7C11/rack3_module14.csv → A_20%/3.A.R5M7C11
[OK] preprocessing_A/3.A.R5M7C11/rack4_module6.csv → A_20%/3.A.R5M7C11
[OK] preprocessing_A/3.A.R5M7C11/rack3_module5.csv → A_20%/3.A.R5M7C11
[OK] preprocessing_A/3.A.R5M7C11/rack1_module9.csv → A_20%/3.A.R5M7C11
[OK] preprocessing_A/3.A.R5M7C11/rack5_module6.csv → A_20%/3.A.R5M7C11
[OK] preprocessing_A/3.A.R5M7C11/rack5_module7.csv → A_20%/3.A.R5M7C11
[

OSError: 197280 requested and 102368 written

In [ ]:
from numba import njit, prange
import os
import glob
import numpy as np
import pandas as pd
from typing import List
from sklearn.impute import KNNImputer
from joblib import Parallel, delayed

# ────────────────────────────────────────────────────────────────
# 0. 설정 파라미터
# ────────────────────────────────────────────────────────────────
SEED: int            = 42          # 고정 시드
MISSING_RATE: float  = 0.2        # 5 % 결측치 비율
VOLT_DROP_THR: float = 3.3         # 3.3 V 미만 → NaN
NAN_DAY_THR: int     = 1400        # 하루 NaN 허용 개수 상한
VALID_VOLT_THR: float = 3.6        # 3.6 V 이상 유효 전압 기준
VALID_COUNT_REQ: int = 400         # 날짜별(분*채널) 유효 샘플 수

# 입력 ESS 폴더 (예시)
ESS_SRC_DIR = 'preprocessing_A/00321804(0002)'
# 출력 루트 폴더 (결측 5 %)
ROOT_OUTPUT_DIR = 'A_20%'

# ────────────────────────────────────────────────────────────────
# KNN 임퓨테이션 (날짜별 병렬)
# ────────────────────────────────────────────────────────────────
imputer = KNNImputer(n_neighbors=5, weights='uniform')

@njit(parallel=True, cache=True)
def _sigmoid_cosine_fill(arr: np.ndarray, interp: np.ndarray, k: float, max_win: int) -> np.ndarray:
    N = arr.size
    out = np.empty_like(arr)
    for i in prange(N):
        if np.isnan(arr[i]):
            # 양쪽 2개 값 탐색
            lv = np.empty(2, np.float64)
            rv = np.empty(2, np.float64)
            lc = rc = 0
            j = i - 1
            while j >= 0 and lc < 2 and (i - j) <= max_win:
                if not np.isnan(arr[j]): lv[lc], lc = arr[j], lc+1
                j -= 1
            j = i + 1
            while j < N and rc < 2 and (j - i) <= max_win:
                if not np.isnan(arr[j]): rv[rc], rc = arr[j], rc+1
                j += 1
            if lc == 2 and rc == 2:
                nl = np.hypot(lv[0], lv[1])
                nr = np.hypot(rv[0], rv[1])
                cos_t = (lv[0]*rv[0] + lv[1]*rv[1]) / (nl * nr)
                w = 1.0 / (1.0 + np.exp(k * cos_t))
                ext = 0.5 * ((2*lv[0] - lv[1]) + (2*rv[0] - rv[1]))
                out[i] = (1 - w) * interp[i] + w * ext
            else:
                out[i] = interp[i]
        else:
            out[i] = arr[i]
    return out

@njit(parallel=True, cache=True)
def fill_all_channels(arr2d: np.ndarray, interp2d: np.ndarray, k: float, max_win: int) -> np.ndarray:
    rows, cols = arr2d.shape
    result = np.empty_like(arr2d)
    for c in prange(cols):
        result[:, c] = _sigmoid_cosine_fill(arr2d[:, c], interp2d[:, c], k, max_win)
    return result

# 워밍업: JIT 컴파일 및 캐시
_dummy = np.zeros((1,1), dtype=np.float64)
_dummy = fill_all_channels(_dummy, _dummy, k=4.0, max_win=1440)


def inject_random_missing(df: pd.DataFrame, cols: List[str], rate: float, seed: int = SEED) -> pd.DataFrame:
    rng = np.random.default_rng(seed)
    mask = rng.random(df[cols].shape) < rate
    df2 = df.copy()
    df2.loc[:, cols] = df2[cols].mask(mask)
    return df2


def preprocess_ess_folder(ess_dir: str, root_out_dir: str):
    # CSV 파일 목록
    csv_list = glob.glob(os.path.join(ess_dir, '*.csv'))
    if not csv_list:
        print(f'[WARN] No CSV in {ess_dir}')
        return

    ess_name = os.path.basename(ess_dir.rstrip('/'))
    out_dir = os.path.join(root_out_dir, ess_name)
    os.makedirs(out_dir, exist_ok=True)

    for path in csv_list:
        df = pd.read_csv(path)
        # 날짜 통합
        if 'colec_dt' not in df.columns:
            if 'clct_dt' in df.columns: df.rename(columns={'clct_dt':'colec_dt'}, inplace=True)
            else:
                print(f'[SKIP] {path} : no date')
                continue
        # 컬럼명 정규화
        rename = {c: f'cell_volt_{int(c.split("_")[-1])}' for c in df.columns if c.startswith('cel_volt_')}
        if rename: df.rename(columns=rename, inplace=True)

        # 타임라인 병합
        df['colec_dt'] = pd.to_datetime(df['colec_dt'])
        t0, t1 = df['colec_dt'].min(), df['colec_dt'].max()
        timeline = pd.DataFrame({'colec_dt': pd.date_range(t0, t1, freq='min')})
        merged = timeline.merge(df, on='colec_dt', how='left')
        if 'Unnamed: 0' in merged.columns: merged.drop('Unnamed: 0', axis=1, inplace=True)
        cols = [c for c in merged.columns if c.startswith('cell_volt_')]
        merged.loc[merged['cell_volt_1'] < VOLT_DROP_THR, cols] = np.nan

        merged['date'] = merged['colec_dt'].dt.date
        bad = merged.groupby('date')[cols].apply(lambda x: (x.isna().sum()>NAN_DAY_THR).any())
        clean = merged[~merged['date'].isin(bad[bad].index)].reset_index(drop=True)

        clean = inject_random_missing(clean, cols, MISSING_RATE)

        # 6. 날짜별 KNN + Sigmoid·Cosine 병렬
        col_idx = [clean.columns.get_loc(c) for c in cols]
        data = clean[cols].to_numpy(dtype=np.float64)
        groups = list(clean.groupby('date').groups.items())  # [(date, idx_list)]

        def process_day(idx_list):
            arr = data[idx_list]
            knn = imputer.fit_transform(arr)
            filled = fill_all_channels(arr, knn, k=4.0, max_win=1440)
            return idx_list, filled

        results = Parallel(n_jobs=-1)(delayed(process_day)(idx) for _, idx in groups)
        for idx_list, filled in results:
            clean.iloc[idx_list, col_idx] = filled.astype(np.float32)

        # 7~9: 유효 날짜 필터 및 저장
        valid = clean.groupby('date')[cols].apply(lambda x: (x>=VALID_VOLT_THR).sum().sum())
        keep = valid[valid>=VALID_COUNT_REQ*len(cols)].index
        df2 = clean[clean['date'].isin(keep)].reset_index(drop=True)
        if df2.empty:
            print(f'[WARN] {path} : no valid dates')
            continue
        first, last = df2['date'].iloc[0], df2['date'].iloc[-1]
        df2 = df2[(df2['date']!=first)&(df2['date']!=last)].reset_index(drop=True)
        if df2.empty:
            print(f'[WARN] {path} : nothing after edge')
            continue

        uniq = df2['date'].unique()
        base = os.path.splitext(os.path.basename(path))[0]
        for c in cols:
            arr = np.stack([df2[df2['date']==d][c].to_numpy(dtype=np.float32) for d in uniq])
            np.save(os.path.join(out_dir, f'{base}_{c}.npy'), arr)
        print(f'[OK] {path} → {out_dir}')

if __name__ == '__main__':
    np.random.seed(SEED)
    os.makedirs(ROOT_OUTPUT_DIR, exist_ok=True)
    preprocess_ess_folder(ESS_SRC_DIR, ROOT_OUTPUT_DIR)


[OK] preprocessing_A/00321804(0002)/rack4_module1.csv → A_20%/00321804(0002)
[OK] preprocessing_A/00321804(0002)/rack16_module5.csv → A_20%/00321804(0002)
[OK] preprocessing_A/00321804(0002)/rack9_module9.csv → A_20%/00321804(0002)
[OK] preprocessing_A/00321804(0002)/rack17_module10.csv → A_20%/00321804(0002)
[OK] preprocessing_A/00321804(0002)/rack15_module9.csv → A_20%/00321804(0002)
[OK] preprocessing_A/00321804(0002)/rack6_module8.csv → A_20%/00321804(0002)
[OK] preprocessing_A/00321804(0002)/rack4_module2.csv → A_20%/00321804(0002)
[OK] preprocessing_A/00321804(0002)/rack9_module1.csv → A_20%/00321804(0002)
[OK] preprocessing_A/00321804(0002)/rack15_module17.csv → A_20%/00321804(0002)
[OK] preprocessing_A/00321804(0002)/rack13_module13.csv → A_20%/00321804(0002)
[OK] preprocessing_A/00321804(0002)/rack18_module14.csv → A_20%/00321804(0002)
[OK] preprocessing_A/00321804(0002)/rack10_module2.csv → A_20%/00321804(0002)
[OK] preprocessing_A/00321804(0002)/rack9_module17.csv → A_20%/00

In [ ]:
from numba import njit, prange
import os
import glob
import numpy as np
import pandas as pd
from typing import List
from sklearn.impute import KNNImputer
from joblib import Parallel, delayed

# ────────────────────────────────────────────────────────────────
# 0. 설정 파라미터
# ────────────────────────────────────────────────────────────────
SEED: int            = 42          # 고정 시드
MISSING_RATE: float  = 0.20        # 5 % 결측치 비율
VOLT_DROP_THR: float = 3.3         # 3.3 V 미만 → NaN
NAN_DAY_THR: int     = 1400        # 하루 NaN 허용 개수 상한
VALID_VOLT_THR: float = 3.6        # 3.6 V 이상 유효 전압 기준
VALID_COUNT_REQ: int = 400         # 날짜별(분*채널) 유효 샘플 수

# 입력 ESS 폴더 (예시)
ESS_SRC_DIR = 'preprocessing_A/00329601(0002)'
# 출력 루트 폴더 (결측 5 %)
ROOT_OUTPUT_DIR = 'A_20%'

# ────────────────────────────────────────────────────────────────
# KNN 임퓨테이션 (날짜별 병렬)
# ────────────────────────────────────────────────────────────────
imputer = KNNImputer(n_neighbors=5, weights='uniform')

@njit(parallel=True, cache=True)
def _sigmoid_cosine_fill(arr: np.ndarray, interp: np.ndarray, k: float, max_win: int) -> np.ndarray:
    N = arr.size
    out = np.empty_like(arr)
    for i in prange(N):
        if np.isnan(arr[i]):
            # 양쪽 2개 값 탐색
            lv = np.empty(2, np.float64)
            rv = np.empty(2, np.float64)
            lc = rc = 0
            j = i - 1
            while j >= 0 and lc < 2 and (i - j) <= max_win:
                if not np.isnan(arr[j]): lv[lc], lc = arr[j], lc+1
                j -= 1
            j = i + 1
            while j < N and rc < 2 and (j - i) <= max_win:
                if not np.isnan(arr[j]): rv[rc], rc = arr[j], rc+1
                j += 1
            if lc == 2 and rc == 2:
                nl = np.hypot(lv[0], lv[1])
                nr = np.hypot(rv[0], rv[1])
                cos_t = (lv[0]*rv[0] + lv[1]*rv[1]) / (nl * nr)
                w = 1.0 / (1.0 + np.exp(k * cos_t))
                ext = 0.5 * ((2*lv[0] - lv[1]) + (2*rv[0] - rv[1]))
                out[i] = (1 - w) * interp[i] + w * ext
            else:
                out[i] = interp[i]
        else:
            out[i] = arr[i]
    return out

@njit(parallel=True, cache=True)
def fill_all_channels(arr2d: np.ndarray, interp2d: np.ndarray, k: float, max_win: int) -> np.ndarray:
    rows, cols = arr2d.shape
    result = np.empty_like(arr2d)
    for c in prange(cols):
        result[:, c] = _sigmoid_cosine_fill(arr2d[:, c], interp2d[:, c], k, max_win)
    return result

# 워밍업: JIT 컴파일 및 캐시
_dummy = np.zeros((1,1), dtype=np.float64)
_dummy = fill_all_channels(_dummy, _dummy, k=4.0, max_win=1440)


def inject_random_missing(df: pd.DataFrame, cols: List[str], rate: float, seed: int = SEED) -> pd.DataFrame:
    rng = np.random.default_rng(seed)
    mask = rng.random(df[cols].shape) < rate
    df2 = df.copy()
    df2.loc[:, cols] = df2[cols].mask(mask)
    return df2


def preprocess_ess_folder(ess_dir: str, root_out_dir: str):
    # CSV 파일 목록
    csv_list = glob.glob(os.path.join(ess_dir, '*.csv'))
    if not csv_list:
        print(f'[WARN] No CSV in {ess_dir}')
        return

    ess_name = os.path.basename(ess_dir.rstrip('/'))
    out_dir = os.path.join(root_out_dir, ess_name)
    os.makedirs(out_dir, exist_ok=True)

    for path in csv_list:
        df = pd.read_csv(path)
        # 날짜 통합
        if 'colec_dt' not in df.columns:
            if 'clct_dt' in df.columns: df.rename(columns={'clct_dt':'colec_dt'}, inplace=True)
            else:
                print(f'[SKIP] {path} : no date')
                continue
        # 컬럼명 정규화
        rename = {c: f'cell_volt_{int(c.split("_")[-1])}' for c in df.columns if c.startswith('cel_volt_')}
        if rename: df.rename(columns=rename, inplace=True)

        # 타임라인 병합
        df['colec_dt'] = pd.to_datetime(df['colec_dt'])
        t0, t1 = df['colec_dt'].min(), df['colec_dt'].max()
        timeline = pd.DataFrame({'colec_dt': pd.date_range(t0, t1, freq='min')})
        merged = timeline.merge(df, on='colec_dt', how='left')
        if 'Unnamed: 0' in merged.columns: merged.drop('Unnamed: 0', axis=1, inplace=True)
        cols = [c for c in merged.columns if c.startswith('cell_volt_')]
        merged.loc[merged['cell_volt_1'] < VOLT_DROP_THR, cols] = np.nan

        merged['date'] = merged['colec_dt'].dt.date
        bad = merged.groupby('date')[cols].apply(lambda x: (x.isna().sum()>NAN_DAY_THR).any())
        clean = merged[~merged['date'].isin(bad[bad].index)].reset_index(drop=True)

        clean = inject_random_missing(clean, cols, MISSING_RATE)

        # 6. 날짜별 KNN + Sigmoid·Cosine 병렬
        col_idx = [clean.columns.get_loc(c) for c in cols]
        data = clean[cols].to_numpy(dtype=np.float64)
        groups = list(clean.groupby('date').groups.items())  # [(date, idx_list)]

        def process_day(idx_list):
            arr = data[idx_list]
            knn = imputer.fit_transform(arr)
            filled = fill_all_channels(arr, knn, k=4.0, max_win=1440)
            return idx_list, filled

        results = Parallel(n_jobs=-1)(delayed(process_day)(idx) for _, idx in groups)
        for idx_list, filled in results:
            clean.iloc[idx_list, col_idx] = filled.astype(np.float32)

        # 7~9: 유효 날짜 필터 및 저장
        valid = clean.groupby('date')[cols].apply(lambda x: (x>=VALID_VOLT_THR).sum().sum())
        keep = valid[valid>=VALID_COUNT_REQ*len(cols)].index
        df2 = clean[clean['date'].isin(keep)].reset_index(drop=True)
        if df2.empty:
            print(f'[WARN] {path} : no valid dates')
            continue
        first, last = df2['date'].iloc[0], df2['date'].iloc[-1]
        df2 = df2[(df2['date']!=first)&(df2['date']!=last)].reset_index(drop=True)
        if df2.empty:
            print(f'[WARN] {path} : nothing after edge')
            continue

        uniq = df2['date'].unique()
        base = os.path.splitext(os.path.basename(path))[0]
        for c in cols:
            arr = np.stack([df2[df2['date']==d][c].to_numpy(dtype=np.float32) for d in uniq])
            np.save(os.path.join(out_dir, f'{base}_{c}.npy'), arr)
        print(f'[OK] {path} → {out_dir}')

if __name__ == '__main__':
    np.random.seed(SEED)
    os.makedirs(ROOT_OUTPUT_DIR, exist_ok=True)
    preprocess_ess_folder(ESS_SRC_DIR, ROOT_OUTPUT_DIR)


[OK] preprocessing_A/00329601(0002)/rack4_module1.csv → A_20%/00329601(0002)
[OK] preprocessing_A/00329601(0002)/rack9_module9.csv → A_20%/00329601(0002)
[OK] preprocessing_A/00329601(0002)/rack15_module9.csv → A_20%/00329601(0002)
[OK] preprocessing_A/00329601(0002)/rack6_module8.csv → A_20%/00329601(0002)
[OK] preprocessing_A/00329601(0002)/rack4_module2.csv → A_20%/00329601(0002)
[OK] preprocessing_A/00329601(0002)/rack9_module1.csv → A_20%/00329601(0002)
[OK] preprocessing_A/00329601(0002)/rack15_module17.csv → A_20%/00329601(0002)
[OK] preprocessing_A/00329601(0002)/rack13_module13.csv → A_20%/00329601(0002)
[OK] preprocessing_A/00329601(0002)/rack10_module2.csv → A_20%/00329601(0002)
[OK] preprocessing_A/00329601(0002)/rack9_module17.csv → A_20%/00329601(0002)
[OK] preprocessing_A/00329601(0002)/rack10_module9.csv → A_20%/00329601(0002)
[OK] preprocessing_A/00329601(0002)/rack2_module17.csv → A_20%/00329601(0002)
[OK] preprocessing_A/00329601(0002)/rack11_module9.csv → A_20%/0032

In [ ]:
from numba import njit, prange
import os
import glob
import numpy as np
import pandas as pd
from typing import List
from sklearn.impute import KNNImputer
from joblib import Parallel, delayed

# ────────────────────────────────────────────────────────────────
# 0. 설정 파라미터
# ────────────────────────────────────────────────────────────────
SEED: int            = 42          # 고정 시드
MISSING_RATE: float  = 0.2        # 5 % 결측치 비율
VOLT_DROP_THR: float = 3.3         # 3.3 V 미만 → NaN
NAN_DAY_THR: int     = 1400        # 하루 NaN 허용 개수 상한
VALID_VOLT_THR: float = 3.6        # 3.6 V 이상 유효 전압 기준
VALID_COUNT_REQ: int = 400         # 날짜별(분*채널) 유효 샘플 수

# 입력 ESS 폴더 (예시)
ESS_SRC_DIR = 'preprocessing_A/00391262(0002)'
# 출력 루트 폴더 (결측 5 %)
ROOT_OUTPUT_DIR = 'A_20%'

# ────────────────────────────────────────────────────────────────
# KNN 임퓨테이션 (날짜별 병렬)
# ────────────────────────────────────────────────────────────────
imputer = KNNImputer(n_neighbors=5, weights='uniform')

@njit(parallel=True, cache=True)
def _sigmoid_cosine_fill(arr: np.ndarray, interp: np.ndarray, k: float, max_win: int) -> np.ndarray:
    N = arr.size
    out = np.empty_like(arr)
    for i in prange(N):
        if np.isnan(arr[i]):
            # 양쪽 2개 값 탐색
            lv = np.empty(2, np.float64)
            rv = np.empty(2, np.float64)
            lc = rc = 0
            j = i - 1
            while j >= 0 and lc < 2 and (i - j) <= max_win:
                if not np.isnan(arr[j]): lv[lc], lc = arr[j], lc+1
                j -= 1
            j = i + 1
            while j < N and rc < 2 and (j - i) <= max_win:
                if not np.isnan(arr[j]): rv[rc], rc = arr[j], rc+1
                j += 1
            if lc == 2 and rc == 2:
                nl = np.hypot(lv[0], lv[1])
                nr = np.hypot(rv[0], rv[1])
                cos_t = (lv[0]*rv[0] + lv[1]*rv[1]) / (nl * nr)
                w = 1.0 / (1.0 + np.exp(k * cos_t))
                ext = 0.5 * ((2*lv[0] - lv[1]) + (2*rv[0] - rv[1]))
                out[i] = (1 - w) * interp[i] + w * ext
            else:
                out[i] = interp[i]
        else:
            out[i] = arr[i]
    return out

@njit(parallel=True, cache=True)
def fill_all_channels(arr2d: np.ndarray, interp2d: np.ndarray, k: float, max_win: int) -> np.ndarray:
    rows, cols = arr2d.shape
    result = np.empty_like(arr2d)
    for c in prange(cols):
        result[:, c] = _sigmoid_cosine_fill(arr2d[:, c], interp2d[:, c], k, max_win)
    return result

# 워밍업: JIT 컴파일 및 캐시
_dummy = np.zeros((1,1), dtype=np.float64)
_dummy = fill_all_channels(_dummy, _dummy, k=4.0, max_win=1440)


def inject_random_missing(df: pd.DataFrame, cols: List[str], rate: float, seed: int = SEED) -> pd.DataFrame:
    rng = np.random.default_rng(seed)
    mask = rng.random(df[cols].shape) < rate
    df2 = df.copy()
    df2.loc[:, cols] = df2[cols].mask(mask)
    return df2


def preprocess_ess_folder(ess_dir: str, root_out_dir: str):
    # CSV 파일 목록
    csv_list = glob.glob(os.path.join(ess_dir, '*.csv'))
    if not csv_list:
        print(f'[WARN] No CSV in {ess_dir}')
        return

    ess_name = os.path.basename(ess_dir.rstrip('/'))
    out_dir = os.path.join(root_out_dir, ess_name)
    os.makedirs(out_dir, exist_ok=True)

    for path in csv_list:
        df = pd.read_csv(path)
        # 날짜 통합
        if 'colec_dt' not in df.columns:
            if 'clct_dt' in df.columns: df.rename(columns={'clct_dt':'colec_dt'}, inplace=True)
            else:
                print(f'[SKIP] {path} : no date')
                continue
        # 컬럼명 정규화
        rename = {c: f'cell_volt_{int(c.split("_")[-1])}' for c in df.columns if c.startswith('cel_volt_')}
        if rename: df.rename(columns=rename, inplace=True)

        # 타임라인 병합
        df['colec_dt'] = pd.to_datetime(df['colec_dt'])
        t0, t1 = df['colec_dt'].min(), df['colec_dt'].max()
        timeline = pd.DataFrame({'colec_dt': pd.date_range(t0, t1, freq='min')})
        merged = timeline.merge(df, on='colec_dt', how='left')
        if 'Unnamed: 0' in merged.columns: merged.drop('Unnamed: 0', axis=1, inplace=True)
        cols = [c for c in merged.columns if c.startswith('cell_volt_')]
        merged.loc[merged['cell_volt_1'] < VOLT_DROP_THR, cols] = np.nan

        merged['date'] = merged['colec_dt'].dt.date
        bad = merged.groupby('date')[cols].apply(lambda x: (x.isna().sum()>NAN_DAY_THR).any())
        clean = merged[~merged['date'].isin(bad[bad].index)].reset_index(drop=True)

        clean = inject_random_missing(clean, cols, MISSING_RATE)

        # 6. 날짜별 KNN + Sigmoid·Cosine 병렬
        col_idx = [clean.columns.get_loc(c) for c in cols]
        data = clean[cols].to_numpy(dtype=np.float64)
        groups = list(clean.groupby('date').groups.items())  # [(date, idx_list)]

        def process_day(idx_list):
            arr = data[idx_list]
            knn = imputer.fit_transform(arr)
            filled = fill_all_channels(arr, knn, k=4.0, max_win=1440)
            return idx_list, filled

        results = Parallel(n_jobs=-1)(delayed(process_day)(idx) for _, idx in groups)
        for idx_list, filled in results:
            clean.iloc[idx_list, col_idx] = filled.astype(np.float32)

        # 7~9: 유효 날짜 필터 및 저장
        valid = clean.groupby('date')[cols].apply(lambda x: (x>=VALID_VOLT_THR).sum().sum())
        keep = valid[valid>=VALID_COUNT_REQ*len(cols)].index
        df2 = clean[clean['date'].isin(keep)].reset_index(drop=True)
        if df2.empty:
            print(f'[WARN] {path} : no valid dates')
            continue
        first, last = df2['date'].iloc[0], df2['date'].iloc[-1]
        df2 = df2[(df2['date']!=first)&(df2['date']!=last)].reset_index(drop=True)
        if df2.empty:
            print(f'[WARN] {path} : nothing after edge')
            continue

        uniq = df2['date'].unique()
        base = os.path.splitext(os.path.basename(path))[0]
        for c in cols:
            arr = np.stack([df2[df2['date']==d][c].to_numpy(dtype=np.float32) for d in uniq])
            np.save(os.path.join(out_dir, f'{base}_{c}.npy'), arr)
        print(f'[OK] {path} → {out_dir}')

if __name__ == '__main__':
    np.random.seed(SEED)
    os.makedirs(ROOT_OUTPUT_DIR, exist_ok=True)
    preprocess_ess_folder(ESS_SRC_DIR, ROOT_OUTPUT_DIR)


[OK] preprocessing_A/00391262(0002)/rack24_module3.csv → A_20%/00391262(0002)
[OK] preprocessing_A/00391262(0002)/rack4_module1.csv → A_20%/00391262(0002)
[OK] preprocessing_A/00391262(0002)/rack16_module5.csv → A_20%/00391262(0002)
[OK] preprocessing_A/00391262(0002)/rack9_module9.csv → A_20%/00391262(0002)
[OK] preprocessing_A/00391262(0002)/rack20_module8.csv → A_20%/00391262(0002)
[OK] preprocessing_A/00391262(0002)/rack26_module1.csv → A_20%/00391262(0002)
[OK] preprocessing_A/00391262(0002)/rack17_module10.csv → A_20%/00391262(0002)
[OK] preprocessing_A/00391262(0002)/rack15_module9.csv → A_20%/00391262(0002)
[OK] preprocessing_A/00391262(0002)/rack6_module8.csv → A_20%/00391262(0002)
[OK] preprocessing_A/00391262(0002)/rack20_module15.csv → A_20%/00391262(0002)
[OK] preprocessing_A/00391262(0002)/rack22_module11.csv → A_20%/00391262(0002)
[OK] preprocessing_A/00391262(0002)/rack28_module5.csv → A_20%/00391262(0002)
[OK] preprocessing_A/00391262(0002)/rack4_module2.csv → A_20%/00

In [ ]:
aaa

In [ ]:
import os
import glob
import numpy as np
import pandas as pd
from numba import njit, prange
from sklearn.impute import KNNImputer
from joblib import Parallel, delayed

# ────────────────────────────────────────────────────────────────
# 설정 파라미터
# ────────────────────────────────────────────────────────────────
SEED = 42                # 고정 시드
MISSING_RATE = 0.2      # 결측치 비율 (5%)
ROOT_OUTPUT_DIR = 'A_20%'  # 최상위 저장 폴더

# ────────────────────────────────────────────────────────────────
# Sigmoid·Cosine Fill 구현 (base 코드와 동일)
# ────────────────────────────────────────────────────────────────
@njit(parallel=True, cache=True)
def _sigmoid_cosine_fill(arr: np.ndarray, interp: np.ndarray, k: float, max_win: int) -> np.ndarray:
    N = arr.size
    out = np.empty_like(arr)
    for i in prange(N):
        if np.isnan(arr[i]):
            lv = np.empty(2, np.float64); rv = np.empty(2, np.float64)
            lc = rc = 0
            j = i - 1
            while j >= 0 and lc < 2 and (i - j) <= max_win:
                if not np.isnan(arr[j]):
                    lv[lc] = arr[j]; lc += 1
                j -= 1
            j = i + 1
            while j < N and rc < 2 and (j - i) <= max_win:
                if not np.isnan(arr[j]):
                    rv[rc] = arr[j]; rc += 1
                j += 1
            if lc == 2 and rc == 2:
                nl = np.hypot(lv[0], lv[1]); nr = np.hypot(rv[0], rv[1])
                cos_t = (lv[0]*rv[0] + lv[1]*rv[1]) / (nl * nr)
                w = 1.0 / (1.0 + np.exp(k * cos_t))
                ext = 0.5 * ((2*lv[0] - lv[1]) + (2*rv[0] - rv[1]))
                out[i] = (1 - w) * interp[i] + w * ext
            else:
                out[i] = interp[i]
        else:
            out[i] = arr[i]
    return out

@njit(parallel=True, cache=True)
def fill_all_channels(arr2d: np.ndarray, interp2d: np.ndarray, k: float, max_win: int) -> np.ndarray:
    rows, cols = arr2d.shape
    result = np.empty_like(arr2d)
    for c in prange(cols):
        result[:, c] = _sigmoid_cosine_fill(arr2d[:, c], interp2d[:, c], k, max_win)
    return result

# KNN Imputer (base 코드와 동일)
imputer = KNNImputer(n_neighbors=5, weights='uniform')


def process_bank_data_2d(df, bank_label, base_filename, output_folder):
    # ... (이전 부분은 변경 없음) ...

    df_copy = df.copy()
    # 1. 시간 컬럼 처리
    if "colec_dt" not in df_copy.columns:
        if "clct_dt" in df_copy.columns:
            df_copy.rename(columns={'clct_dt': 'colec_dt'}, inplace=True)
        else:
            print(f"{base_filename} - bank {bank_label}: 시간 컬럼 없음, 스킵")
            return

    # 2. 전압 컬럼 이름 정규화
    if not any(col.startswith("cell_volt_") for col in df_copy.columns):
        rename_dict = {}
        for col in df_copy.columns:
            if col.startswith("cel_volt_"):
                num = col.replace("cel_volt_", "")
                if num.isdigit():
                    rename_dict[col] = f"cell_volt_{int(num)}"
        if rename_dict:
            df_copy.rename(columns=rename_dict, inplace=True)

    # 3. datetime 변환 및 재인덱싱
    df_copy["colec_dt"] = pd.to_datetime(df_copy["colec_dt"])
    start, end = df_copy["colec_dt"].min(), df_copy["colec_dt"].max()
    full_idx = pd.DataFrame({"colec_dt": pd.date_range(start, end, freq='min')})
    merged = pd.merge(full_idx, df_copy, on="colec_dt", how="left")
    merged = merged.groupby("colec_dt").first().reset_index()

    # 4. 불필요 컬럼 제거
    if "Unnamed: 0" in merged.columns:
        merged.drop(columns=["Unnamed: 0"], inplace=True)
    # volt_cols 정의
    volt_cols = [c for c in merged.columns if c.startswith("cell_volt_")]

    # 5. 전압 < 3.3V → NaN
    merged.loc[merged[volt_cols[0]] < 3.3, volt_cols] = np.nan

    # 날짜 컬럼 추가 및 과도 결측일 제거
    merged["date"] = merged["colec_dt"].dt.date
    nan_thr = 1400
    nan_counts = merged.groupby("date")[volt_cols].apply(lambda df: df.isna().sum())
    to_drop = (nan_counts > nan_thr).any(axis=1)
    merged = merged[~merged["date"].isin(to_drop[to_drop].index)].reset_index(drop=True)

    # 6. ★ 결측치 5% 무작위 주입 ★
    rng = np.random.default_rng(SEED)
    mask = rng.random(merged[volt_cols].shape) < MISSING_RATE
    merged.loc[:, volt_cols] = merged[volt_cols].mask(mask)

    # 7. 날짜별 KNN + Sigmoid·Cosine 병렬 처리 (base 코드와 동일)
    col_idx = [merged.columns.get_loc(c) for c in volt_cols]
    data = merged[volt_cols].to_numpy(dtype=np.float64)
    groups = list(merged.groupby('date').groups.items())

    def process_day(idx_list):
        arr = data[idx_list]
        knn = imputer.fit_transform(arr)
        filled = fill_all_channels(arr, knn, 4.0, 1440)
        return idx_list, filled

    results = Parallel(n_jobs=-1)(delayed(process_day)(idx) for _, idx in groups)
    for idx_list, filled in results:
        merged.iloc[idx_list, col_idx] = filled.astype(np.float32)

    # 8. 유효일 필터링
    vt_thr, req_count = 3.6, 400*len(volt_cols)
    vc = merged.groupby("date")[volt_cols].apply(lambda df: (df>=vt_thr).sum().sum())
    keep = vc[vc>=req_count].index
    merged = merged[merged["date"].isin(keep)].reset_index(drop=True)

    # 9. 가장 앞/뒤 하루 제거
    dates = merged["date"].unique()
    if len(dates) <= 2:
        print(f"{base_filename} - bank {bank_label}: 유효일 부족")
        return
    merged = merged[~merged["date"].isin([dates[0], dates[-1]])].reset_index(drop=True)
    unique_dates = merged["date"].unique()

    # 10. 채널별 2D 배열 생성 및 저장 (float32)
    for volt in volt_cols:
        X = np.empty([len(unique_dates), 1440], dtype=np.float32)
        valid_idx = []
        for i, d in enumerate(unique_dates):
            day = merged[merged["date"]==d]
            series = day[volt]
            if series.shape[0] != 1440:
                print(f"{base_filename}, bank{bank_label}, {d}, {volt}: {series.shape[0]} rows")
                continue
            valid_idx.append(i)
            X[i] = series.to_numpy(dtype=np.float32)
        if not valid_idx:
            print(f"{base_filename} - bank {bank_label}: {volt} 유효 데이터 없음")
            continue
        X = X[valid_idx]
        out_path = os.path.join(output_folder, f"{base_filename}_{volt}.npy")
        np.save(out_path, X)
        print(f"Saved: {out_path} shape={X.shape}")

# ────────────────────────────────────────────────────────────────
# 사용 예시
# ────────────────────────────────────────────────────────────────
if __name__ == '__main__':
    np.random.seed(SEED)  # 전역 시드 고정

    input_folder = 'preprocessing_A/00370960(0002)'
    ess_files = glob.glob(os.path.join(input_folder, '*.csv'))
    base = os.path.basename(input_folder)

    # ★ A_5%/<base>_1, A_5%/<base>_2 폴더 생성 ★
    bank1_out = os.path.join(ROOT_OUTPUT_DIR, f"{base}_1")
    bank2_out = os.path.join(ROOT_OUTPUT_DIR, f"{base}_2")
    for p in [bank1_out, bank2_out]:
        os.makedirs(p, exist_ok=True)

    for path in ess_files:
        df = pd.read_csv(path)
        name = os.path.splitext(os.path.basename(path))[0]
        df1 = df[df['bsc_fg_no']==1]
        df2 = df[df['bsc_fg_no']==2]
        if not df1.empty:
            process_bank_data_2d(df1, 1, name, bank1_out)
        if not df2.empty:
            process_bank_data_2d(df2, 2, name, bank2_out)


Saved: A_20%/00370960(0002)_1/rack4_module1_cell_volt_1.npy shape=(34, 1440)
Saved: A_20%/00370960(0002)_1/rack4_module1_cell_volt_2.npy shape=(34, 1440)
Saved: A_20%/00370960(0002)_1/rack4_module1_cell_volt_3.npy shape=(34, 1440)
Saved: A_20%/00370960(0002)_1/rack4_module1_cell_volt_4.npy shape=(34, 1440)
Saved: A_20%/00370960(0002)_1/rack4_module1_cell_volt_5.npy shape=(34, 1440)
Saved: A_20%/00370960(0002)_1/rack4_module1_cell_volt_6.npy shape=(34, 1440)
Saved: A_20%/00370960(0002)_1/rack4_module1_cell_volt_7.npy shape=(34, 1440)
Saved: A_20%/00370960(0002)_1/rack4_module1_cell_volt_8.npy shape=(34, 1440)
Saved: A_20%/00370960(0002)_1/rack4_module1_cell_volt_9.npy shape=(34, 1440)
Saved: A_20%/00370960(0002)_1/rack4_module1_cell_volt_10.npy shape=(34, 1440)
Saved: A_20%/00370960(0002)_1/rack4_module1_cell_volt_11.npy shape=(34, 1440)
Saved: A_20%/00370960(0002)_1/rack4_module1_cell_volt_12.npy shape=(34, 1440)
Saved: A_20%/00370960(0002)_1/rack4_module1_cell_volt_13.npy shape=(34, 1

In [ ]:
import os
import glob
import numpy as np
import pandas as pd
from numba import njit, prange
from sklearn.impute import KNNImputer
from joblib import Parallel, delayed

# ────────────────────────────────────────────────────────────────
# 설정 파라미터
# ────────────────────────────────────────────────────────────────
SEED = 42                # 고정 시드
MISSING_RATE = 0.2      # 결측치 비율 (5%)
ROOT_OUTPUT_DIR = 'A_20%'  # 최상위 저장 폴더

# ────────────────────────────────────────────────────────────────
# Sigmoid·Cosine Fill 구현 (base 코드와 동일)
# ────────────────────────────────────────────────────────────────
@njit(parallel=True, cache=True)
def _sigmoid_cosine_fill(arr: np.ndarray, interp: np.ndarray, k: float, max_win: int) -> np.ndarray:
    N = arr.size
    out = np.empty_like(arr)
    for i in prange(N):
        if np.isnan(arr[i]):
            lv = np.empty(2, np.float64); rv = np.empty(2, np.float64)
            lc = rc = 0
            j = i - 1
            while j >= 0 and lc < 2 and (i - j) <= max_win:
                if not np.isnan(arr[j]):
                    lv[lc] = arr[j]; lc += 1
                j -= 1
            j = i + 1
            while j < N and rc < 2 and (j - i) <= max_win:
                if not np.isnan(arr[j]):
                    rv[rc] = arr[j]; rc += 1
                j += 1
            if lc == 2 and rc == 2:
                nl = np.hypot(lv[0], lv[1]); nr = np.hypot(rv[0], rv[1])
                cos_t = (lv[0]*rv[0] + lv[1]*rv[1]) / (nl * nr)
                w = 1.0 / (1.0 + np.exp(k * cos_t))
                ext = 0.5 * ((2*lv[0] - lv[1]) + (2*rv[0] - rv[1]))
                out[i] = (1 - w) * interp[i] + w * ext
            else:
                out[i] = interp[i]
        else:
            out[i] = arr[i]
    return out

@njit(parallel=True, cache=True)
def fill_all_channels(arr2d: np.ndarray, interp2d: np.ndarray, k: float, max_win: int) -> np.ndarray:
    rows, cols = arr2d.shape
    result = np.empty_like(arr2d)
    for c in prange(cols):
        result[:, c] = _sigmoid_cosine_fill(arr2d[:, c], interp2d[:, c], k, max_win)
    return result

# KNN Imputer (base 코드와 동일)
imputer = KNNImputer(n_neighbors=5, weights='uniform')


def process_bank_data_2d(df, bank_label, base_filename, output_folder):
    # ... (이전 부분은 변경 없음) ...

    df_copy = df.copy()
    # 1. 시간 컬럼 처리
    if "colec_dt" not in df_copy.columns:
        if "clct_dt" in df_copy.columns:
            df_copy.rename(columns={'clct_dt': 'colec_dt'}, inplace=True)
        else:
            print(f"{base_filename} - bank {bank_label}: 시간 컬럼 없음, 스킵")
            return

    # 2. 전압 컬럼 이름 정규화
    if not any(col.startswith("cell_volt_") for col in df_copy.columns):
        rename_dict = {}
        for col in df_copy.columns:
            if col.startswith("cel_volt_"):
                num = col.replace("cel_volt_", "")
                if num.isdigit():
                    rename_dict[col] = f"cell_volt_{int(num)}"
        if rename_dict:
            df_copy.rename(columns=rename_dict, inplace=True)

    # 3. datetime 변환 및 재인덱싱
    df_copy["colec_dt"] = pd.to_datetime(df_copy["colec_dt"])
    start, end = df_copy["colec_dt"].min(), df_copy["colec_dt"].max()
    full_idx = pd.DataFrame({"colec_dt": pd.date_range(start, end, freq='min')})
    merged = pd.merge(full_idx, df_copy, on="colec_dt", how="left")
    merged = merged.groupby("colec_dt").first().reset_index()

    # 4. 불필요 컬럼 제거
    if "Unnamed: 0" in merged.columns:
        merged.drop(columns=["Unnamed: 0"], inplace=True)
    # volt_cols 정의
    volt_cols = [c for c in merged.columns if c.startswith("cell_volt_")]

    # 5. 전압 < 3.3V → NaN
    merged.loc[merged[volt_cols[0]] < 3.3, volt_cols] = np.nan

    # 날짜 컬럼 추가 및 과도 결측일 제거
    merged["date"] = merged["colec_dt"].dt.date
    nan_thr = 1400
    nan_counts = merged.groupby("date")[volt_cols].apply(lambda df: df.isna().sum())
    to_drop = (nan_counts > nan_thr).any(axis=1)
    merged = merged[~merged["date"].isin(to_drop[to_drop].index)].reset_index(drop=True)

    # 6. ★ 결측치 5% 무작위 주입 ★
    rng = np.random.default_rng(SEED)
    mask = rng.random(merged[volt_cols].shape) < MISSING_RATE
    merged.loc[:, volt_cols] = merged[volt_cols].mask(mask)

    # 7. 날짜별 KNN + Sigmoid·Cosine 병렬 처리 (base 코드와 동일)
    col_idx = [merged.columns.get_loc(c) for c in volt_cols]
    data = merged[volt_cols].to_numpy(dtype=np.float64)
    groups = list(merged.groupby('date').groups.items())

    def process_day(idx_list):
        arr = data[idx_list]
        knn = imputer.fit_transform(arr)
        filled = fill_all_channels(arr, knn, 4.0, 1440)
        return idx_list, filled

    results = Parallel(n_jobs=-1)(delayed(process_day)(idx) for _, idx in groups)
    for idx_list, filled in results:
        merged.iloc[idx_list, col_idx] = filled.astype(np.float32)

    # 8. 유효일 필터링
    vt_thr, req_count = 3.6, 400*len(volt_cols)
    vc = merged.groupby("date")[volt_cols].apply(lambda df: (df>=vt_thr).sum().sum())
    keep = vc[vc>=req_count].index
    merged = merged[merged["date"].isin(keep)].reset_index(drop=True)

    # 9. 가장 앞/뒤 하루 제거
    dates = merged["date"].unique()
    if len(dates) <= 2:
        print(f"{base_filename} - bank {bank_label}: 유효일 부족")
        return
    merged = merged[~merged["date"].isin([dates[0], dates[-1]])].reset_index(drop=True)
    unique_dates = merged["date"].unique()

    # 10. 채널별 2D 배열 생성 및 저장 (float32)
    for volt in volt_cols:
        X = np.empty([len(unique_dates), 1440], dtype=np.float32)
        valid_idx = []
        for i, d in enumerate(unique_dates):
            day = merged[merged["date"]==d]
            series = day[volt]
            if series.shape[0] != 1440:
                print(f"{base_filename}, bank{bank_label}, {d}, {volt}: {series.shape[0]} rows")
                continue
            valid_idx.append(i)
            X[i] = series.to_numpy(dtype=np.float32)
        if not valid_idx:
            print(f"{base_filename} - bank {bank_label}: {volt} 유효 데이터 없음")
            continue
        X = X[valid_idx]
        out_path = os.path.join(output_folder, f"{base_filename}_{volt}.npy")
        np.save(out_path, X)
        print(f"Saved: {out_path} shape={X.shape}")

# ────────────────────────────────────────────────────────────────
# 사용 예시
# ────────────────────────────────────────────────────────────────
if __name__ == '__main__':
    np.random.seed(SEED)  # 전역 시드 고정

    input_folder = 'preprocessing_A/00454547(0002)'
    ess_files = glob.glob(os.path.join(input_folder, '*.csv'))
    base = os.path.basename(input_folder)

    # ★ A_5%/<base>_1, A_5%/<base>_2 폴더 생성 ★
    bank1_out = os.path.join(ROOT_OUTPUT_DIR, f"{base}_1")
    bank2_out = os.path.join(ROOT_OUTPUT_DIR, f"{base}_2")
    for p in [bank1_out, bank2_out]:
        os.makedirs(p, exist_ok=True)

    for path in ess_files:
        df = pd.read_csv(path)
        name = os.path.splitext(os.path.basename(path))[0]
        df1 = df[df['bsc_fg_no']==1]
        df2 = df[df['bsc_fg_no']==2]
        if not df1.empty:
            process_bank_data_2d(df1, 1, name, bank1_out)
        if not df2.empty:
            process_bank_data_2d(df2, 2, name, bank2_out)


Saved: A_20%/00454547(0002)_1/rack4_module1_cell_volt_1.npy shape=(37, 1440)
Saved: A_20%/00454547(0002)_1/rack4_module1_cell_volt_2.npy shape=(37, 1440)
Saved: A_20%/00454547(0002)_1/rack4_module1_cell_volt_3.npy shape=(37, 1440)
Saved: A_20%/00454547(0002)_1/rack4_module1_cell_volt_4.npy shape=(37, 1440)
Saved: A_20%/00454547(0002)_1/rack4_module1_cell_volt_5.npy shape=(37, 1440)
Saved: A_20%/00454547(0002)_1/rack4_module1_cell_volt_6.npy shape=(37, 1440)
Saved: A_20%/00454547(0002)_1/rack4_module1_cell_volt_7.npy shape=(37, 1440)
Saved: A_20%/00454547(0002)_1/rack4_module1_cell_volt_8.npy shape=(37, 1440)
Saved: A_20%/00454547(0002)_1/rack4_module1_cell_volt_9.npy shape=(37, 1440)
Saved: A_20%/00454547(0002)_1/rack4_module1_cell_volt_10.npy shape=(37, 1440)
Saved: A_20%/00454547(0002)_1/rack4_module1_cell_volt_11.npy shape=(37, 1440)
Saved: A_20%/00454547(0002)_1/rack4_module1_cell_volt_12.npy shape=(37, 1440)
Saved: A_20%/00454547(0002)_1/rack4_module1_cell_volt_13.npy shape=(37, 1

In [ ]:
bbb

In [ ]:
import os
import glob
import numpy as np
import pandas as pd
from numba import njit, prange
from sklearn.impute import KNNImputer
from joblib import Parallel, delayed

# ────────────────────────────────────────────────────────────────
# 설정 파라미터
# ────────────────────────────────────────────────────────────────
SEED = 42                # 고정 시드
MISSING_RATE = 0.2      # 결측치 비율 (5%)
ROOT_OUTPUT_DIR = 'A_20%'  # 최상위 저장 폴더

# 입력 폴더 및 CSV 파일 리스트
ess_name = 'preprocessing_A/2.A.R7M17C6'
ess_folder = os.path.basename(ess_name.rstrip('/'))
ess_list = glob.glob(os.path.join(ess_name, '*.csv'))

# ★ 출력 루트 폴더 생성: A_5%/<ess_folder> ★
output_root = os.path.join(ROOT_OUTPUT_DIR, ess_folder)
os.makedirs(output_root, exist_ok=True)

# ────────────────────────────────────────────────────────────────
# Sigmoid·Cosine Fill 구현 (base 코드와 동일)
# ────────────────────────────────────────────────────────────────
@njit(parallel=True, cache=True)
def _sigmoid_cosine_fill(arr: np.ndarray, interp: np.ndarray, k: float, max_win: int) -> np.ndarray:
    N = arr.size
    out = np.empty_like(arr)
    for i in prange(N):
        if np.isnan(arr[i]):
            lv = np.empty(2, np.float64); rv = np.empty(2, np.float64)
            lc = rc = 0
            j = i - 1
            while j >= 0 and lc < 2 and (i - j) <= max_win:
                if not np.isnan(arr[j]): lv[lc] = arr[j]; lc += 1
                j -= 1
            j = i + 1
            while j < N and rc < 2 and (j - i) <= max_win:
                if not np.isnan(arr[j]): rv[rc] = arr[j]; rc += 1
                j += 1
            if lc == 2 and rc == 2:
                nl = np.hypot(lv[0], lv[1]); nr = np.hypot(rv[0], rv[1])
                cos_t = (lv[0]*rv[0] + lv[1]*rv[1]) / (nl * nr)
                w = 1.0 / (1.0 + np.exp(k * cos_t))
                ext = 0.5 * ((2*lv[0] - lv[1]) + (2*rv[0] - rv[1]))
                out[i] = (1 - w) * interp[i] + w * ext
            else:
                out[i] = interp[i]
        else:
            out[i] = arr[i]
    return out

# 채널별 처리 유틸
@njit(parallel=True, cache=True)
def fill_all_channels(arr2d: np.ndarray, interp2d: np.ndarray, k: float, max_win: int) -> np.ndarray:
    rows, cols = arr2d.shape
    result = np.empty_like(arr2d)
    for c in prange(cols):
        result[:, c] = _sigmoid_cosine_fill(arr2d[:, c], interp2d[:, c], k, max_win)
    return result

# KNN Imputer
imputer = KNNImputer(n_neighbors=5, weights='uniform')

for ess_path in ess_list:
    df = pd.read_csv(ess_path)
    if 'colec_dt' not in df.columns:
        if 'clct_dt' in df.columns:
            df.rename(columns={'clct_dt':'colec_dt'}, inplace=True)
        else: continue
    df['colec_dt'] = pd.to_datetime(df['colec_dt'])
    df.drop_duplicates(subset='colec_dt', inplace=True)

    # 타임라인 merge
    t0, t1 = df['colec_dt'].min(), df['colec_dt'].max()
    timeline = pd.DataFrame({'colec_dt': pd.date_range(t0, t1, freq='min')})
    merged_df = pd.merge(timeline, df, on='colec_dt', how='left')
    if 'Unnamed: 0' in merged_df.columns:
        merged_df.drop(columns=['Unnamed: 0'], inplace=True)

    # 컬럼명 정규화
    rename_dict = {c: f"cell_volt_{int(c.split('_')[-1])}" for c in merged_df.columns if c.startswith('cel_volt_')}
    if rename_dict:
        merged_df.rename(columns=rename_dict, inplace=True)
    volt_cols = [c for c in merged_df.columns if c.startswith('cell_volt_')]
    if not volt_cols: continue

    merged_df.loc[merged_df[volt_cols[0]] < 3.3, volt_cols] = np.nan
    merged_df['date'] = merged_df['colec_dt'].dt.date

    # 과도 결측일 제거
    nan_thr = 1400
    nan_counts = merged_df.groupby('date')[volt_cols].apply(lambda df: df.isna().sum())
    drop_dates = nan_counts.gt(nan_thr).any(axis=1)
    merged_df = merged_df[~merged_df['date'].isin(drop_dates[drop_dates].index)].reset_index(drop=True)

    # 5% 무작위 결측 주입
    rng = np.random.default_rng(SEED)
    mask = rng.random(merged_df[volt_cols].shape) < MISSING_RATE
    merged_df.loc[:, volt_cols] = merged_df[volt_cols].mask(mask)

    # 날짜별 KNN + Sigmoid·Cosine 보간
    col_idx = [merged_df.columns.get_loc(c) for c in volt_cols]
    data = merged_df[volt_cols].to_numpy(dtype=np.float64)
    groups = list(merged_df.groupby('date').groups.items())

    def process_day(idx_list):
        arr = data[idx_list]
        knn_arr = imputer.fit_transform(arr)
        filled = fill_all_channels(arr, knn_arr, 4.0, 1440)
        return idx_list, filled

    results = Parallel(n_jobs=-1)(delayed(process_day)(idx) for _, idx in groups)
    for idx_list, filled in results:
        merged_df.iloc[idx_list, col_idx] = filled.astype(np.float32)

    # 유효일 필터링 및 첫/마지막 제외
    vt_thr, req_cnt = 3.6, 400 * len(volt_cols)
    valid = merged_df.groupby('date')[volt_cols].apply(lambda df: (df>=vt_thr).sum().sum())
    merged_df = merged_df[merged_df['date'].isin(valid[valid>=req_cnt].index)].reset_index(drop=True)
    dates = merged_df['date'].unique()
    if len(dates) <= 2: continue
    merged_df = merged_df[~merged_df['date'].isin([dates[0], dates[-1]])].reset_index(drop=True)

    # 저장
    base = os.path.splitext(os.path.basename(ess_path))[0]
    for volt in volt_cols:
        X = np.stack([merged_df[merged_df['date']==d][volt].to_numpy(dtype=np.float32)
                      for d in merged_df['date'].unique()])
        np.save(os.path.join(output_root, f"{base}_{volt}.npy"), X)
        print(f"Saved: {base}_{volt}.npy shape={X.shape}")


Saved: rack4_module1_cell_volt_1.npy shape=(75, 1440)
Saved: rack4_module1_cell_volt_2.npy shape=(75, 1440)
Saved: rack4_module1_cell_volt_3.npy shape=(75, 1440)
Saved: rack4_module1_cell_volt_4.npy shape=(75, 1440)
Saved: rack4_module1_cell_volt_5.npy shape=(75, 1440)
Saved: rack4_module1_cell_volt_6.npy shape=(75, 1440)
Saved: rack4_module1_cell_volt_7.npy shape=(75, 1440)
Saved: rack4_module1_cell_volt_8.npy shape=(75, 1440)
Saved: rack4_module1_cell_volt_9.npy shape=(75, 1440)
Saved: rack4_module1_cell_volt_10.npy shape=(75, 1440)
Saved: rack4_module1_cell_volt_11.npy shape=(75, 1440)
Saved: rack4_module1_cell_volt_12.npy shape=(75, 1440)
Saved: rack4_module1_cell_volt_13.npy shape=(75, 1440)
Saved: rack4_module1_cell_volt_14.npy shape=(75, 1440)
Saved: rack16_module5_cell_volt_1.npy shape=(74, 1440)
Saved: rack16_module5_cell_volt_2.npy shape=(74, 1440)
Saved: rack16_module5_cell_volt_3.npy shape=(74, 1440)
Saved: rack16_module5_cell_volt_4.npy shape=(74, 1440)
Saved: rack16_modul

/tmp/ipykernel_363634/342413678.py:118: RuntimeWarning: overflow encountered in cast
  merged_df.iloc[idx_list, col_idx] = filled.astype(np.float32)


Saved: rack17_module10_cell_volt_1.npy shape=(73, 1440)
Saved: rack17_module10_cell_volt_2.npy shape=(73, 1440)
Saved: rack17_module10_cell_volt_3.npy shape=(73, 1440)
Saved: rack17_module10_cell_volt_4.npy shape=(73, 1440)
Saved: rack17_module10_cell_volt_5.npy shape=(73, 1440)
Saved: rack17_module10_cell_volt_6.npy shape=(73, 1440)
Saved: rack17_module10_cell_volt_7.npy shape=(73, 1440)
Saved: rack17_module10_cell_volt_8.npy shape=(73, 1440)
Saved: rack17_module10_cell_volt_9.npy shape=(73, 1440)
Saved: rack17_module10_cell_volt_10.npy shape=(73, 1440)
Saved: rack17_module10_cell_volt_11.npy shape=(73, 1440)
Saved: rack17_module10_cell_volt_12.npy shape=(73, 1440)
Saved: rack17_module10_cell_volt_13.npy shape=(73, 1440)
Saved: rack17_module10_cell_volt_14.npy shape=(73, 1440)


/tmp/ipykernel_363634/342413678.py:118: RuntimeWarning: overflow encountered in cast
  merged_df.iloc[idx_list, col_idx] = filled.astype(np.float32)


Saved: rack15_module9_cell_volt_1.npy shape=(73, 1440)
Saved: rack15_module9_cell_volt_2.npy shape=(73, 1440)
Saved: rack15_module9_cell_volt_3.npy shape=(73, 1440)
Saved: rack15_module9_cell_volt_4.npy shape=(73, 1440)
Saved: rack15_module9_cell_volt_5.npy shape=(73, 1440)
Saved: rack15_module9_cell_volt_6.npy shape=(73, 1440)
Saved: rack15_module9_cell_volt_7.npy shape=(73, 1440)
Saved: rack15_module9_cell_volt_8.npy shape=(73, 1440)
Saved: rack15_module9_cell_volt_9.npy shape=(73, 1440)
Saved: rack15_module9_cell_volt_10.npy shape=(73, 1440)
Saved: rack15_module9_cell_volt_11.npy shape=(73, 1440)
Saved: rack15_module9_cell_volt_12.npy shape=(73, 1440)
Saved: rack15_module9_cell_volt_13.npy shape=(73, 1440)
Saved: rack15_module9_cell_volt_14.npy shape=(73, 1440)
Saved: rack6_module8_cell_volt_1.npy shape=(74, 1440)
Saved: rack6_module8_cell_volt_2.npy shape=(74, 1440)
Saved: rack6_module8_cell_volt_3.npy shape=(74, 1440)
Saved: rack6_module8_cell_volt_4.npy shape=(74, 1440)
Saved: ra

...

Saved: rack5_module15_cell_volt_1.npy shape=(73, 1440)
Saved: rack5_module15_cell_volt_2.npy shape=(73, 1440)
Saved: rack5_module15_cell_volt_3.npy shape=(73, 1440)
Saved: rack5_module15_cell_volt_4.npy shape=(73, 1440)
Saved: rack5_module15_cell_volt_5.npy shape=(73, 1440)
Saved: rack5_module15_cell_volt_6.npy shape=(73, 1440)
Saved: rack5_module15_cell_volt_7.npy shape=(73, 1440)
Saved: rack5_module15_cell_volt_8.npy shape=(73, 1440)
Saved: rack5_module15_cell_volt_9.npy shape=(73, 1440)
Saved: rack5_module15_cell_volt_10.npy shape=(73, 1440)
Saved: rack5_module15_cell_volt_11.npy shape=(73, 1440)
Saved: rack5_module15_cell_volt_12.npy shape=(73, 1440)
Saved: rack5_module15_cell_volt_13.npy shape=(73, 1440)
Saved: rack5_module15_cell_volt_14.npy shape=(73, 1440)


/tmp/ipykernel_363634/342413678.py:118: RuntimeWarning: overflow encountered in cast
  merged_df.iloc[idx_list, col_idx] = filled.astype(np.float32)


Saved: rack12_module8_cell_volt_1.npy shape=(74, 1440)
Saved: rack12_module8_cell_volt_2.npy shape=(74, 1440)
Saved: rack12_module8_cell_volt_3.npy shape=(74, 1440)
Saved: rack12_module8_cell_volt_4.npy shape=(74, 1440)
Saved: rack12_module8_cell_volt_5.npy shape=(74, 1440)
Saved: rack12_module8_cell_volt_6.npy shape=(74, 1440)
Saved: rack12_module8_cell_volt_7.npy shape=(74, 1440)
Saved: rack12_module8_cell_volt_8.npy shape=(74, 1440)
Saved: rack12_module8_cell_volt_9.npy shape=(74, 1440)
Saved: rack12_module8_cell_volt_10.npy shape=(74, 1440)
Saved: rack12_module8_cell_volt_11.npy shape=(74, 1440)
Saved: rack12_module8_cell_volt_12.npy shape=(74, 1440)
Saved: rack12_module8_cell_volt_13.npy shape=(74, 1440)
Saved: rack12_module8_cell_volt_14.npy shape=(74, 1440)
Saved: rack10_module5_cell_volt_1.npy shape=(72, 1440)
Saved: rack10_module5_cell_volt_2.npy shape=(72, 1440)
Saved: rack10_module5_cell_volt_3.npy shape=(72, 1440)
Saved: rack10_module5_cell_volt_4.npy shape=(72, 1440)
Saved

/tmp/ipykernel_363634/342413678.py:118: RuntimeWarning: overflow encountered in cast
  merged_df.iloc[idx_list, col_idx] = filled.astype(np.float32)


Saved: rack10_module12_cell_volt_1.npy shape=(74, 1440)
Saved: rack10_module12_cell_volt_2.npy shape=(74, 1440)
Saved: rack10_module12_cell_volt_3.npy shape=(74, 1440)
Saved: rack10_module12_cell_volt_4.npy shape=(74, 1440)
Saved: rack10_module12_cell_volt_5.npy shape=(74, 1440)
Saved: rack10_module12_cell_volt_6.npy shape=(74, 1440)
Saved: rack10_module12_cell_volt_7.npy shape=(74, 1440)
Saved: rack10_module12_cell_volt_8.npy shape=(74, 1440)
Saved: rack10_module12_cell_volt_9.npy shape=(74, 1440)
Saved: rack10_module12_cell_volt_10.npy shape=(74, 1440)
Saved: rack10_module12_cell_volt_11.npy shape=(74, 1440)
Saved: rack10_module12_cell_volt_12.npy shape=(74, 1440)
Saved: rack10_module12_cell_volt_13.npy shape=(74, 1440)
Saved: rack10_module12_cell_volt_14.npy shape=(74, 1440)
Saved: rack13_module1_cell_volt_1.npy shape=(73, 1440)
Saved: rack13_module1_cell_volt_2.npy shape=(73, 1440)
Saved: rack13_module1_cell_volt_3.npy shape=(73, 1440)
Saved: rack13_module1_cell_volt_4.npy shape=(7